## ReadCSV and preprocess

In [ ]:
library(dplyr)
library(readr)
library(rjags)

In [ ]:
# Read your data
df <- read_csv("data.csv")

# Create team indices (important for modeling)
teams <- sort(unique(c(df$home_team, df$away_team)))
team_index <- setNames(seq_along(teams), teams)

df <- df %>%
  mutate(
    H = team_index[home_team],
    A = team_index[away_team]
  )


## Model in JAGS for free throws

In [ ]:
model_string_pois <- "
model {
  for (i in 1:N) {
    # Home
    HFT[i] ~ dpois(thetaH[i])
    log(thetaH[i]) <- att[H[i]] + def[A[i]] + c + home
    
    # Away
    AFT[i] ~ dpois(thetaA[i])
    log(thetaA[i]) <- att[A[i]] + def[H[i]] + c
  }

  # Priors
  for (j in 1:K) {
    att[j] ~ dnorm(mu_att, tau_att)
    def[j] ~ dnorm(mu_def, tau_def)
  }

  mu_att ~ dnorm(0, 0.0001)
  mu_def ~ dnorm(0, 0.0001)
  tau_att ~ dgamma(0.1, 0.1)
  tau_def ~ dgamma(0.1, 0.1)

  c ~ dnorm(0, 0.0001)
  home ~ dnorm(0, 0.0001) T(0,)

  # Identifiability constraints
  att[K+1] <- -sum(att[1:(K-1)])
  def[K+1] <- -sum(def[1:(K-1)]) 
}
"

## Prepare data for rjags

In [ ]:
# Assume you've already processed your DataFrame as above
data_list <- list(
  N = nrow(df),
  K = length(teams)-1,
  HFT = df$ftm_home,
  AFT = df$ftm_away,
  H = df$H,
  A = df$A
)

# Run model
model <- jags.model(textConnection(model_string_pois), data = data_list, n.chains = 3)
update(model, 5000)  # Burn-in

samples_pois <- coda.samples(model, variable.names = c("att", "def", "home", "c"), n.iter = 1000)
summary(samples_pois)


In [ ]:
coda::gelman.diag(samples_pois)  # > 1.1, problem with posterior

In [ ]:
par(bg = 'white')
plot(samples)